# 딥 러닝을 이용한 자연어 처리 입문

## 정제(Cleaning)과 정규화(Normalization)
- **정제(cleaning)** : 갖고 있는 코퍼스로부터 노이즈 데이터를 제거한다.
- **정규화(normalization)** : 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어 준다.

## 정규화 기법
- 정규화의 지향점은 언제나 갖고 있는 코퍼스로부터 복잡성을 줄이는 것이다.

### 길이가 짧은 단어(Removing words with very a short length)

In [ ]:
# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제
import re

text = "I was wondering if anyone out there could enlighten me on this car."
shortword = re.compile(r'\W*\b\w{1,2}\b')

print(shortword.sub('', text))

 was wondering anyone out there could enlighten this car.


### 표제어 추출(Lemmatization)
- **표제어(Lemma)**는 한글로는 '표제어' 또는 '기본 사전형 단어' 정도의 의미이다.
- 표제어 추출은 단어들로부터 표제어를 찾아가는 과정이다. 가령 영어의 am, are, is의 표제어는 be이다.
- 표제어 추출을 하는 가장 섬세한 방법은 단어의 형태론적 파싱을 먼저 진행하는 것이다.
- 형태소는 어간(stem)과 접사(affix)로 나뉘며, **형태론적 파싱**은 이 두 가지 요소를 분리하는 작업을 말한다.

#### nltk : WordNetLemmatizer
- nltk의 표제어 추출 도구
- `n.lemmatize('단어', '품사')`로 특정 단어에 대한 품사를 알려줄 수 있다. 예를 들면 `n.lemmatize('dies', 'v')`

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer

n = WordNetLemmatizer()
words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print([n.lemmatize(w) for w in words])

['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


- 표제어 추출은 어간 추출(stemming)과는 달리 단어의 형태가 보존된다.
- 그럼에도 dy나 ha 같은 단어가 출력되는데 lemmatizer가 품사 정보를 알아야 정확한 결과를 낼 수 있기 때문이다.

In [ ]:
n.lemmatize('dies', 'v')

'die'

In [ ]:
n.lemmatize('watched', 'v')

'watch'

In [ ]:
n.lemmatize('has', 'v')

'have'

- 표제어 추출은 문맥을 고려하며, 수행했을 때의 결과는 해당 단어의 품사 정보를 보존한다.
- 어간 추출을 수행한 결과는 품사 정보가 보존되지 않는다. (POS tag를 고려하지 않는다.)

### 어간 추출(Stemming)
- 어간(Stem)을 추출하는 작업을 **어간 추출(stemming)**이라고 한다.
- 단순 규칙에 의거한 것이므로 사전에 없는 단어들도 포함될 수 있다.

#### 포터 알고리즘(Porter Algorithm)
- 어간 추출 알고리즘 중 하나
- `from nltk.stem import PorterStemmer`
- 어간 추출 속도는 표제어 추출보다 일반적으로 빠른데, 포터 어간 추출기는 정밀하게 설계되어 정확도가 높으므로 영어 자연어 처리에서 어간 추출을 하고자 한다면 가장 준수한 선택이다.

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

s = PorterStemmer()
text="This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."
words=word_tokenize(text)

print(words)

['This', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', "'s", 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things', '--', 'names', 'and', 'heights', 'and', 'soundings', '--', 'with', 'the', 'single', 'exception', 'of', 'the', 'red', 'crosses', 'and', 'the', 'written', 'notes', '.']


In [4]:
print([s.stem(w) for w in words])

['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


In [5]:
words=['formalize', 'allowance', 'electricical']

print([s.stem(w) for w in words])

['formal', 'allow', 'electric']


- 포터 알고리즘의 어간 추출 규칙
 - ALIZE → AL
 - ANCE → 제거
 - ICAL → IC
- Porter 알고리즘의 상세 규칙은 마틴 포터의 홈페이지에서 확인할 수 있다.

#### 랭커스터 스태머(Lancaster Stemmer) 알고리즘
- 포터 알고리즘과 비교해 보자.

In [6]:
# 포터 알고리즘으로 stemming
from nltk.stem import PorterStemmer

s=PorterStemmer()
words=['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print([s.stem(w) for w in words])

['polici', 'do', 'organ', 'have', 'go', 'love', 'live', 'fli', 'die', 'watch', 'ha', 'start']


In [7]:
# 랭커스터 알고리즘으로 stemming
from nltk.stem import LancasterStemmer

l=LancasterStemmer()
words=['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print([l.stem(w) for w in words])

['policy', 'doing', 'org', 'hav', 'going', 'lov', 'liv', 'fly', 'die', 'watch', 'has', 'start']


- 두 스태머가 다른 알고리즘을 사용하므로 코퍼스에 적합한 스태머를 선택해야 한다.
- 규칙 기반 알고리즘은 과일반화로 인해 문제가 될 수 있다.
- organization을 organ으로 만든 포터을 예로 들 수 있다.